In [ ]:
!pip install onnxruntime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 4.5 MB/s eta 0:00:00


In [ ]:
!pip install onnx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 16.7 MB/s eta 0:00:00


In [ ]:
import onnxruntime as ort

In [ ]:
import os

In [ ]:
import numpy as np
import cv2
import onnxruntime
import numpy as np
from PIL import Image
import torch

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [ ]:
image_dir = "/content/gdrive/My Drive/test_images"
#image_dir = "/content/gdrive/MyDrive/comp_test"

In [ ]:
"""from scipy.special import softmax
import torchvision.transforms as transforms
from PIL import Image

sess = ort.InferenceSession("/content/gdrive/My Drive/first_trial.onnx")   #[man, woman]
#sess = ort.InferenceSession("/content/gdrive/MyDrive/components_new.onnx")
#image_dir = "/content/gdrive/My Drive/test_images"

# Define the same transform as in your training code
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Iterate over images in directory
for image_name in os.listdir(image_dir):
    # Load image
    img = cv2.imread(os.path.join(image_dir, image_name))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = Image.fromarray(img)
    img = transform(img)

    # Add batch dimension
    img_batch = np.expand_dims(img, axis=0)

    # Run the model (modify 'input_name' and 'output_name' accordingly)
    input_name = sess.get_inputs()[0].name
    output_name = sess.get_outputs()[0].name
    result = sess.run([output_name], {input_name: img_batch})

    # Apply softmax to convert logits to probabilities
    proba = softmax(result[0], axis=1)

    # Print result
    print(f"Result for {image_name}: {proba}")  #[m,w]: m,m,m,w,m,m,m,w,m,m,m,m,w,m,w"""

'from scipy.special import softmax\nimport torchvision.transforms as transforms\nfrom PIL import Image\n\nsess = ort.InferenceSession("/content/gdrive/My Drive/first_trial.onnx")   #[man, woman]\n#sess = ort.InferenceSession("/content/gdrive/MyDrive/components_new.onnx") \n#image_dir = "/content/gdrive/My Drive/test_images"\n\n# Define the same transform as in your training code\ntransform = transforms.Compose([\n    transforms.Resize((224, 224)),\n    transforms.ToTensor(),\n    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),\n])\n\n# Iterate over images in directory\nfor image_name in os.listdir(image_dir):\n    # Load image\n    img = cv2.imread(os.path.join(image_dir, image_name))\n    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)\n    img = Image.fromarray(img)\n    img = transform(img)\n\n    # Add batch dimension\n    img_batch = np.expand_dims(img, axis=0)\n\n    # Run the model (modify \'input_name\' and \'output_name\' accordingly)\n    input_name = 

In [ ]:
import onnxruntime
import numpy as np
import os
from PIL import Image
import matplotlib.pyplot as plt

# Load the ONNX model
#onnx_model_path = '/content/gdrive/MyDrive/components_new.onnx'
onnx_model_path ='/content/gdrive/My Drive/first_trial.onnx'
sess = onnxruntime.InferenceSession(onnx_model_path)

# Function to preprocess an image
def preprocess_image(image_path):
    image = Image.open(image_path).convert("RGB")
    image = image.resize((224, 224))
    image = np.array(image).astype(np.float32) / 255.0
    image = (image - [0.485, 0.456, 0.406]) / [0.229, 0.224, 0.225]
    image = np.transpose(image, (2, 0, 1))
    image = np.expand_dims(image, axis=0)
    return image.astype(np.float32)  # Ensure float32 data type

# Function to make predictions
def predict_gender(image_path):
    input_image = preprocess_image(image_path)
    input_name = sess.get_inputs()[0].name
    output_name = sess.get_outputs()[0].name

    # Run the ONNX model
    result = sess.run([output_name], {input_name: input_image})

    # Apply sigmoid to convert logits to probabilities
    probabilities = 1 / (1 + np.exp(-result[0]))

    class_names = ['Male', 'Female']
    #class_names = ['Resistor','Capacitor']
    predicted_class = class_names[np.argmax(probabilities)]

    return predicted_class, probabilities

# Path to the folder containing images
#image_folder_path = '/content/gdrive/MyDrive/comp_test'  # Specify the path to your test images
image_dir = "/content/gdrive/My Drive/test_images"

# Display predictions for each image in the folder
for filename in os.listdir(image_folder_path):
    if filename.endswith(('.jpg', '.jpeg', '.png')):
        image_path = os.path.join(image_folder_path, filename)

        # Make predictions
        predicted_class, probabilities = predict_gender(image_path)
        #predicted_class, probabilities = predict_component(image_path)

        # Display the image and prediction
        img = Image.open(image_path)
        plt.imshow(img)
        plt.axis('off')
        plt.title(f'Prediction: {predicted_class} (Male: {float(probabilities[0][0]):.2f}, Female: {float(probabilities[0][1]):.2f})')
        #plt.title(f'Prediction: {predicted_class} (Resistor: {float(probabilities[0][0]):.2f}, Capacitor: {float(probabilities[0][1]):.2f})')
        plt.show()
